In [1]:
import pandas as pd
import numpy as np

In [2]:
notes = pd.read_csv('../NOTEEVENTS.csv')

C:\Miniconda\envs\py37tf2\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
notes.shape

(2083180, 11)

In [4]:
icd = pd.read_csv('../DIAGNOSES_ICD.csv')

In [5]:
icd.shape

(651047, 5)

In [6]:
notes.head()

,ROW_ID,SUBJECT_ID,HADM_ID,CHARTDATE,CHARTTIME,STORETIME,CATEGORY,DESCRIPTION,CGID,ISERROR,TEXT
0,174,22532,167853.0,2151-08-04,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2151-7-16**] Dischar...
1,175,13702,107527.0,2118-06-14,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2118-6-2**] Discharg...
2,176,13702,167118.0,2119-05-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2119-5-4**] D...
3,177,13702,196489.0,2124-08-18,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2124-7-21**] ...
4,178,26880,135453.0,2162-03-25,NaN,NaN,Discharge summary,Report,NaN,NaN,Admission Date: [**2162-3-3**] D...


In [7]:
icd.head()

,ROW_ID,SUBJECT_ID,HADM_ID,SEQ_NUM,ICD9_CODE
0,1297,109,172335,1.0,40301
1,1298,109,172335,2.0,486
2,1299,109,172335,3.0,58281
3,1300,109,172335,4.0,5855
4,1301,109,172335,5.0,4254


In [8]:
notes = notes.merge(icd, left_on=['SUBJECT_ID', 'HADM_ID'], right_on=['SUBJECT_ID','HADM_ID'])

In [9]:
notes.shape

(25736048, 14)

In [10]:
notes = notes[['TEXT','ICD9_CODE']]

In [11]:
notes

,TEXT,ICD9_CODE
0,Admission Date: [**2151-7-16**] Dischar...,01193
1,Admission Date: [**2151-7-16**] Dischar...,4254
2,Admission Date: [**2151-7-16**] Dischar...,42731
3,Admission Date: [**2151-7-16**] Dischar...,2639
4,Admission Date: [**2151-7-16**] Dischar...,2762
...,...,...
25736043,Neonatology\nBaby Girl [**Known lastname 1672*...,V053
25736044,Neonatology\nBaby Girl [**Known lastname 1672*...,V290
25736045,NPN NICU\nTerm female admitted to NICU for sep...,V3001
25736046,NPN NICU\nTerm female admitted to NICU for sep...,V053


# Create File for Labels

In [12]:
labels = notes['ICD9_CODE'].unique()

In [13]:
labels.shape

(6968,)

In [14]:
pd.DataFrame(labels).to_csv('./labels.txt', index=False, header=False)

# Create Multi-class Labels

In [15]:
notes.head()

,TEXT,ICD9_CODE
0,Admission Date: [**2151-7-16**] Dischar...,01193
1,Admission Date: [**2151-7-16**] Dischar...,4254
2,Admission Date: [**2151-7-16**] Dischar...,42731
3,Admission Date: [**2151-7-16**] Dischar...,2639
4,Admission Date: [**2151-7-16**] Dischar...,2762


In [16]:
notes = notes.astype(str)

In [17]:
# Keep top 500 codes
#top_codes = set(notes["ICD9_CODE"].value_counts().nlargest(n=500).index)
# Keep top 100 codes
top_codes = set(notes["ICD9_CODE"].value_counts().nlargest(n=100).index)

In [18]:
print((top_codes))

{'40391', '78959', '5185', '45829', '4168', '2749', '5601', 'V290', '4240', '77981', '76524', '49390', '7470', '25000', '4280', '2760', '4254', '4241', '2761', '5770', '0389', '77081', '5856', '5789', '2639', '76503', '53081', 'V502', '2449', '76526', '570', '32723', '2720', 'V5861', '2762', '2930', '5990', '9971', '7742', 'V053', '486', '2875', '70703', 'V3001', '42789', '78039', '99811', '2859', '2767', '5119', '412', '4271', '76514', 'V4582', '7707', '51884', '48241', '00845', 'E8798', '77181', '4275', '2724', 'V3000', '78551', '76527', 'V1582', '5180', '496', '40390', '99662', '5849', '42731', '77989', '99859', '41071', '2851', '769', '99731', '7766', '4589', '311', '99592', '7907', '36221', '73300', '3051', '5859', '7793', '78552', 'V3101', '76525', '2763', '42732', 'V5867', '41401', '5845', '4019', '5070', 'V4581', '51881'}


In [20]:
pd.DataFrame(top_codes).to_csv('../top_codes.txt', index=False, header=False)

In [19]:
#len(top_codes)
print(sum(notes["ICD9_CODE"].value_counts().nlargest(n=100)))

12554672


In [20]:
sum(notes["ICD9_CODE"].value_counts().nlargest(n=100)) / notes.shape[0]

0.48782439324017424

In [21]:
notes.loc[~notes["ICD9_CODE"].isin(top_codes), 'ICD9_CODE'] = "OTHER"

In [25]:
sum(notes["ICD9_CODE"].value_counts().nlargest(n=1)) / notes.shape[0] # Due to Other

0.5121756067598258

In [22]:
from sklearn.preprocessing import MultiLabelBinarizer

In [23]:
mlb = MultiLabelBinarizer(sparse_output=True)

In [24]:
mlb.fit([list(notes['ICD9_CODE'].unique())])

MultiLabelBinarizer(sparse_output=True)

In [25]:
mlb.classes_

array(['00845', '0389', '2449', '25000', '2639', '2720', '2724', '2749',
       '2760', '2761', '2762', '2763', '2767', '2851', '2859', '2875',
       '2930', '3051', '311', '32723', '36221', '4019', '40390', '40391',
       '41071', '412', '41401', '4168', '4240', '4241', '4254', '4271',
       '42731', '42732', '4275', '42789', '4280', '45829', '4589',
       '48241', '486', '49390', '496', '5070', '5119', '5180', '5185',
       '51881', '51884', '53081', '5601', '570', '5770', '5789', '5845',
       '5849', '5856', '5859', '5990', '70703', '73300', '7470', '76503',
       '76514', '76524', '76525', '76526', '76527', '769', '7707',
       '77081', '77181', '7742', '7766', '7793', '77981', '77989',
       '78039', '78551', '78552', '78959', '7907', '99592', '99662',
       '9971', '99731', '99811', '99859', 'E8798', 'OTHER', 'V053',
       'V1582', 'V290', 'V3000', 'V3001', 'V3101', 'V4581', 'V4582',
       'V502', 'V5861', 'V5867'], dtype=object)

In [26]:
len(mlb.classes_)

101

In [32]:
set(mlb.classes_) - set(top_codes)

{'OTHER'}

In [28]:
pd.DataFrame(list(mlb.classes_)).to_csv('./top_codes_orderbylabel.txt', index=False, header=False)

# Groupby text create lists of labels

In [29]:
notes = notes.groupby('TEXT').ICD9_CODE.apply(list).reset_index()

In [30]:
test = mlb.transform(notes['ICD9_CODE'])

In [31]:
test

<1801952x101 sparse matrix of type '<class 'numpy.int32'>'
	with 14004526 stored elements in Compressed Sparse Row format>

In [32]:
temp = pd.DataFrame(test.todense())

In [33]:
# Join notes to one-hot-encodings based on index
notes = pd.concat([notes, temp], axis=1)

In [34]:
notes.head()

,TEXT,ICD9_CODE,0,1,2,3,4,5,6,7,...,91,92,93,94,95,96,97,98,99,100
0,\n\n\n,"[OTHER, 70703, OTHER, OTHER, OTHER, OTHER, OTH...",1,1,1,1,0,1,1,1,...,1,0,0,0,0,1,1,0,1,0
1,\n\n\n\n,"[OTHER, 51881, OTHER, OTHER, 4589, OTHER, OTHE...",0,1,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,0,0
2,\n\n\n\n\n,"[2762, 78552, 4280, OTHER, 99592, 0389, 486, 5...",1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
3,\n\n\n\n\n\n\n,"[51881, OTHER, OTHER, 2767, 5849, 42731, OTHER...",0,0,0,1,0,0,0,0,...,0,1,0,1,1,0,0,1,0,0
4,\n\n\n\n\n\n\n\n,"[OTHER, 51881, OTHER, OTHER, OTHER, 5849, OTHE...",0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
del temp

In [36]:
del test

In [43]:
# Start with column 0
notes['labels'] = notes[[notes.columns[2]]].values.tolist()

In [49]:

#notes.head()
a = notes['labels'][0]#.head()
print(len(a))

101


In [44]:
notes['labels'] = notes[notes.columns[2:-1]].values.tolist()
#notes['labels'] = notes[notes.loc['0':'100']].values.tolist()

In [42]:
notes.head()

,TEXT,ICD9_CODE,0,1,2,3,4,5,6,7,...,92,93,94,95,96,97,98,99,100,labels
0,\n\n\n,"[OTHER, 70703, OTHER, OTHER, OTHER, OTHER, OTH...",1,1,1,1,0,1,1,1,...,0,0,0,0,1,1,0,1,0,"[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,\n\n\n\n,"[OTHER, 51881, OTHER, OTHER, 4589, OTHER, OTHE...",0,1,1,0,0,1,0,0,...,1,0,1,0,0,0,0,0,0,"[0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, ..."
2,\n\n\n\n\n,"[2762, 78552, 4280, OTHER, 99592, 0389, 486, 5...",1,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,1,"[1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, ..."
3,\n\n\n\n\n\n\n,"[51881, OTHER, OTHER, 2767, 5849, 42731, OTHER...",0,0,0,1,0,0,0,0,...,1,0,1,1,0,0,1,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, ..."
4,\n\n\n\n\n\n\n\n,"[OTHER, 51881, OTHER, OTHER, OTHER, 5849, OTHE...",0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
# notes['labels'] = notes.apply(lambda x: x['labels'] + [x[col]], 1)

In [ ]:
# for col in notes.columns[3:-1]:
#     notes['labels'] = notes.apply(lambda x: x['labels'] + [x[col]], 1)
#     notes.drop(col, axis=1, inplace=True)

# Drop All Columns That are Unused

In [50]:
notes = notes[['TEXT','labels']]

In [51]:
notes = notes.rename({'TEXT': 'text'}, axis=1)

# Create Train, Dev, & Test Files

In [52]:
tmp = notes.sample(frac=0.3)
train, validate, test = np.split(tmp, [int(.8*len(tmp)), int(0.9*len(tmp))])

In [53]:
train.shape

(432468, 2)

In [54]:
validate.shape

(54059, 2)

In [48]:
test.shape

(54059, 2)

In [55]:
train.to_csv('./train21Aug27.tsv', index=False, sep='\t')

In [56]:
validate.to_csv('../validate21Aug27.tsv', index=False, sep='\t')

In [57]:
test.to_csv('../test21Aug27.tsv', index=False, sep='\t')

In [58]:
a = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1]
print(len(a))

101
